In [1]:
from ase.io import read, write, Trajectory
import numpy as np

In [2]:
def CoM(clus):
    """
    Support function to set the origin of the cluster at the centre of the mass
    """
    (cx, cy, cz) = clus.get_center_of_mass()
    new_xyz = []
    for i, a in enumerate(clus):
        x, y, z = a.position
        x -= cx
        y -= cy
        z -= cz
        new_xyz.append((x, y, z))
    clus.set_positions(new_xyz)
    return clus

In [3]:
def checkSimilar(clus1, clus2):

    """Check whether two clusters are similar or not by comparing their moments of inertia"""
    Inertia1 = clus1.get_moments_of_inertia()
    Inertia2 = clus2.get_moments_of_inertia()
    # print(Inertia1, Inertia2, 'diff: ', Inertia1-Inertia2)

    tol = 0.01
    if (
        Inertia1[0] * (1 - tol) <= Inertia2[0] <= Inertia1[0] * (1 + tol)
        and Inertia1[1] * (1 - tol) <= Inertia2[1] <= Inertia1[1] * (1 + tol)
        and Inertia1[2] * (1 - tol) <= Inertia2[2] <= Inertia1[2] * (1 + tol)
    ):
        similar = True
    else:
        similar = False

    return similar

In [4]:
ini_gen = 0
end_gen = 25

In [5]:
ene_tol = 0.02
dist_tol = 0.02

num_check_similar = 0
diff_dist_abs_sum_list = []

non_similar_clus = []
similar_clus = []
non_similar_clus_ene_list = []
similar_clus_ene_list = []
non_similar_clus_diff_dist = []
similar_clus_diff_dist = []

n_traj1_list = []
n_traj2_list = []

for k in range(ini_gen, end_gen):
    file1  = 'mut_after_relax_gen'+str(k+1)+'.traj'
    file2  = 'vasp_mut_after_relax_gen'+str(k+1)+'.traj'

    traj1 = Trajectory(file1)
    traj2 = Trajectory(file2)
    
    n_traj1 = len(traj1)
    n_traj2 = len(traj2)
    n_traj1_list.append(n_traj1)
    n_traj2_list.append(n_traj2)
    
    for i in range(len(traj1)):
        num_check_similar = num_check_similar + 1
        clus1 = traj1[i]
        clus2 = traj2[i]
        
        ene1 = clus1.get_potential_energy()
        ene2 = clus2.get_potential_energy()
        diff_ene = ene1 - ene2
        
        dist1 = clus1.get_all_distances() 
        dist2 = clus2.get_all_distances()
        diff_dist = dist1 - dist2
        diff_dist_abs = np.absolute(diff_dist)
        diff_dist_avg = np.sum(diff_dist_abs) / len(diff_dist)**2
        
        inertia1 = clus1.get_moments_of_inertia()
        inertia2 = clus2.get_moments_of_inertia()
        diff_inertia = inertia1 - inertia2
              
        if checkSimilar(clus1, clus2) == True:
            if diff_dist_avg > dist_tol:
                if abs(diff_ene) > ene_tol:
                    non_similar_clus.append((k,i))
                    non_similar_clus_ene_list.append(diff_ene)
                    non_similar_clus_diff_dist.append(diff_dist_avg)
                    print((k, i),diff_dist_avg, diff_ene, diff_inertia )
                else:
                    similar_clus.append((k,i))
                    similar_clus_ene_list.append(diff_ene)
                    similar_clus_diff_dist.append(diff_dist_avg)
                    #print((k, i),diff_dist_avg, diff_ene, diff_inertia )  
                    
            else:
                similar_clus.append((k,i))
                similar_clus_ene_list.append(diff_ene)
                similar_clus_diff_dist.append(diff_dist_avg)
                #print((k, i),diff_dist_avg, diff_ene, diff_inertia )  
                
        else:
            if diff_dist_avg < dist_tol:
                if abs(diff_ene) < ene_tol:
                    similar_clus.append((k,i))
                    similar_clus_ene_list.append(diff_ene)
                    similar_clus_diff_dist.append(diff_dist_avg)
                    #print((k, i),diff_dist_avg, diff_ene, diff_inertia )
                else: 
                    non_similar_clus.append((k,i))
                    non_similar_clus_ene_list.append(diff_ene)
                    non_similar_clus_diff_dist.append(diff_dist_avg)
                    print((k, i),diff_dist_avg, diff_ene, diff_inertia )
            else:
                non_similar_clus.append((k,i))
                non_similar_clus_ene_list.append(diff_ene)
                non_similar_clus_diff_dist.append(diff_dist_avg)
                print((k, i),diff_dist_avg, diff_ene, diff_inertia )

print(len(non_similar_clus))
print(len(similar_clus))
print(num_check_similar)

#print(sorted(non_similar_clus_ene_list))
#print(non_similar_clus)
#print(sorted(non_similar_clus_ene_list))
#print(sorted(non_similar_clus_diff_dist))

print(n_traj1_list)
print(n_traj2_list)

(0, 1) 0.40130078258749224 -0.2986535700000026 [1733.36390138 -299.10532632  444.63047517]
(0, 3) 0.21838222588702882 -0.29103733000000176 [629.40838695 913.74097864 315.62964603]
(0, 4) 0.025874140189824965 0.004222120000001439 [-12.59170983  42.46332814  16.76661908]
(0, 5) 0.2876770673594497 -0.5461188900000025 [ 835.34671594 -112.98351393  -37.72592408]
(0, 6) 0.2902886056924373 -0.7975327700000001 [1473.54023897 -139.55573216 -886.24541088]
(0, 7) 0.46534471797182775 -1.1372637700000006 [ 913.6773736   569.16307015 -642.83321039]
(0, 8) 0.49594248791284695 -1.4020428199999984 [  850.11039287    99.89941337 -1459.96991878]
(1, 1) 0.30858559986297196 -0.8307559799999993 [ 534.82516163  138.31589196 -953.46059018]
(1, 4) 0.35187833972031124 -0.6496474999999968 [   82.43109715  -100.46064862 -1281.03851469]
(1, 5) 0.3488669758463745 -0.6452355500000024 [   34.28952672  -180.00403845 -1260.9202389 ]
(1, 6) 0.4335490049600034 -0.8114873399999993 [1300.24221054    6.88909542 -783.1914960

(17, 2) 0.4848477457936522 -0.9459986700000016 [ 1108.46596983 -2432.90243372 -1260.573659  ]
(17, 3) 4.200746523503018 51.59813937 [ 1792.12360053 21248.38608243 23703.7714967 ]
(17, 6) 0.9766457623209847 -1.3026072100000015 [ 1315.56965152 -2318.41927297 -4630.16419227]
(17, 7) 0.7153000663071101 0.9965127000000003 [-898.40271438 1380.09228162 4176.25119849]
(17, 8) 1.2832285055676729 -3.7905304499999986 [ 1249.32002064 -7607.75683817 -7122.28971368]
(17, 9) 0.3999952109238232 -0.28347405999999964 [1153.05174948  450.41567475  842.23661881]
(18, 0) 0.40981787058822594 -1.0862993199999984 [1607.88245857  473.06037903 -505.51809612]
(18, 1) 0.2795279976947833 -0.1432708500000004 [  88.24479578 -486.75984918 -500.27063493]
(18, 2) 0.5040685103579546 -0.7579739199999977 [ 1275.0769392    273.49937213 -1153.08989348]
(18, 3) 0.6853812128549642 -2.1402258200000013 [1665.32661452  821.17979026   64.71220639]
(18, 5) 0.6937951078620048 1.190136540000001 [-876.805121    701.42810292 2558.8649

In [7]:
filename="clus_Cu4Au4.log"

parent_calls_list = []
with open(filename, 'r') as fh:
    for line in fh:
        if line.startswith("Parent"):
            line = line.strip()
            words = line.split(':')
            words[1] = words[1].lstrip()
            parent_calls_list.append(words[1])
#print(parent_calls_list)
#print((parent_calls_list[0]))

item_split = []
for item in parent_calls_list:
    print( item)
    item_split.append(item.split(","))
#print(item_split)

#print(item_split[0])


[10, 30, 6, 38, 17, 38, 26, 20, 16]
[15, 35, 6, 11, 19, 15, 17, 22, 6]
[52, 33, 23, 23, 53, 24, 25, 6, 21, 22]
[6, 14, 7, 17, 16, 47, 21, 23, 37, 15]
[35, 8, 18, 28, 24, 19, 12, 35, 17, 48]
[31, 33, 14, 10, 30, 50, 31, 31, 12, 10]
[36, 6, 16, 46, 27, 16, 19, 11, 24, 38]
[15, 20, 12, 17, 13, 10, 39, 38, 31, 6]
[34, 43, 8, 38, 45, 27, 27, 18, 21, 20]
[11, 23, 19, 9, 17, 21, 20, 26, 32, 19]
[33, 46, 6, 30, 32, 14, 20, 33, 20, 6]
[7, 6, 9, 24, 19, 28, 29, 24]
[8, 25, 28, 16, 16, 11, 35, 59, 27]
[6, 38, 21, 23, 6, 13, 22, 28, 18]
[29, 30, 6, 27, 16, 6, 63, 50, 35]
[29, 11, 8, 23, 20, 27, 44, 20, 39, 27]
[23, 22, 23, 6, 28, 25, 21, 25, 6, 13]
[26, 23, 27, 9, 6, 27, 24, 19, 41, 22]
[20, 32, 24, 47, 14, 45, 16, 57, 23, 12]
[8, 33, 15, 11, 23, 18, 16, 22, 30]
[18, 17, 37, 29, 6, 23, 41, 22, 42, 27]
[14, 13, 17, 16, 30, 26, 20, 24, 32, 22]
[46, 6, 24, 51, 14, 22, 20, 33, 74, 23]
[21, 15, 10, 6, 6, 26, 7, 14, 6, 14]
[29, 16, 24, 26, 38, 27, 32, 16, 13, 30]


In [8]:
new_vasp_calls = []
len_new_vasp_calls = []
for i in range(len(item_split)):
    k=len(item_split[i])
    new_int_list = []
    
    a = list(item_split[i][0].lstrip())
    del a[0] 
    a = int(''.join(a))
    new_int_list.append(a)
    #print(a)
    for k in range(1,(len(item_split[i])-1)):
        b = int(item_split[i][k])
        new_int_list.append(b)
        #print(b)  
    c = list(item_split[i][-1].lstrip())
    del c[-1]
    c = int(''.join(c))
    new_int_list.append(c)
    #print(c)
    #print(new_int_list)
    len_new_list = len(new_int_list)
    new_vasp_calls.append(new_int_list)
    len_new_vasp_calls.append(len_new_list)
print(new_vasp_calls)
print(len(new_vasp_calls))
print(len_new_vasp_calls)
print(sum(len_new_vasp_calls))

[[10, 30, 6, 38, 17, 38, 26, 20, 16], [15, 35, 6, 11, 19, 15, 17, 22, 6], [52, 33, 23, 23, 53, 24, 25, 6, 21, 22], [6, 14, 7, 17, 16, 47, 21, 23, 37, 15], [35, 8, 18, 28, 24, 19, 12, 35, 17, 48], [31, 33, 14, 10, 30, 50, 31, 31, 12, 10], [36, 6, 16, 46, 27, 16, 19, 11, 24, 38], [15, 20, 12, 17, 13, 10, 39, 38, 31, 6], [34, 43, 8, 38, 45, 27, 27, 18, 21, 20], [11, 23, 19, 9, 17, 21, 20, 26, 32, 19], [33, 46, 6, 30, 32, 14, 20, 33, 20, 6], [7, 6, 9, 24, 19, 28, 29, 24], [8, 25, 28, 16, 16, 11, 35, 59, 27], [6, 38, 21, 23, 6, 13, 22, 28, 18], [29, 30, 6, 27, 16, 6, 63, 50, 35], [29, 11, 8, 23, 20, 27, 44, 20, 39, 27], [23, 22, 23, 6, 28, 25, 21, 25, 6, 13], [26, 23, 27, 9, 6, 27, 24, 19, 41, 22], [20, 32, 24, 47, 14, 45, 16, 57, 23, 12], [8, 33, 15, 11, 23, 18, 16, 22, 30], [18, 17, 37, 29, 6, 23, 41, 22, 42, 27], [14, 13, 17, 16, 30, 26, 20, 24, 32, 22], [46, 6, 24, 51, 14, 22, 20, 33, 74, 23], [21, 15, 10, 6, 6, 26, 7, 14, 6, 14], [29, 16, 24, 26, 38, 27, 32, 16, 13, 30]]
25
[9, 9, 10, 

In [9]:
print(len(similar_clus))
print(similar_clus)

74
[(0, 0), (0, 2), (1, 0), (1, 2), (1, 3), (2, 7), (2, 8), (3, 0), (3, 1), (3, 2), (3, 5), (3, 7), (4, 1), (5, 0), (5, 9), (6, 1), (6, 5), (6, 9), (7, 0), (7, 1), (7, 3), (7, 5), (7, 9), (8, 2), (8, 5), (8, 9), (9, 0), (9, 1), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (10, 2), (10, 9), (11, 0), (11, 1), (11, 2), (12, 0), (12, 4), (12, 8), (13, 0), (13, 3), (13, 4), (13, 5), (14, 2), (14, 5), (15, 1), (15, 2), (16, 3), (16, 6), (16, 7), (16, 8), (16, 9), (17, 4), (17, 5), (18, 4), (19, 0), (19, 2), (19, 3), (20, 0), (20, 4), (20, 5), (21, 0), (21, 1), (21, 2), (21, 9), (22, 1), (22, 4), (23, 3), (23, 4), (23, 6), (23, 8), (24, 1)]


In [10]:
file_vasp="results.log"

vasp_calls = []
vasp_calls_list = []
with open(file_vasp, 'r') as fh:
    total_images = []
    for line in fh:       
        if "VASP Calls" in line:
            line = line.strip()
            words = line.split(':')
            words[1] = int(words[1].lstrip())
            vasp_calls.append(words[1])
        if "Total images in the trajectory" in line:
            line = line.strip()
            img_num = line.split(':')
            img_num[1] = int(img_num[1].lstrip())
            total_images.append(img_num[1])

print(vasp_calls)
#print(len(vasp_calls))
#print(total_images)
#print(len(total_images))
#print(sum(total_images))
print('\n')

kk = 0
for i in total_images:
    traj_calls = []
    for j in range(kk, kk+i):
        traj_calls.append(vasp_calls[j])
    vasp_calls_list.append(traj_calls)
    kk = kk + i

print(vasp_calls_list)

[24, 3, 9, 27, 21, 14, 51, 16, 41, 11, 51, 7, 84, 72, 92, 30, 25, 13, 59, 77, 49, 18, 17, 14, 55, 7, 16, 75, 9, 43, 11, 14, 22, 58, 19, 13, 14, 15, 58, 40, 45, 69, 60, 34, 22, 36, 21, 74, 63, 52, 16, 26, 14, 87, 40, 22, 48, 29, 45, 8, 22, 88, 15, 52, 54, 67, 23, 42, 40, 40, 18, 165, 20, 25, 81, 49, 16, 17, 35, 103, 31, 42, 48, 74, 13, 32, 32, 15, 42, 99, 52, 50, 15, 17, 18, 66, 44, 61, 102, 51, 7, 63, 50, 23, 18, 18, 60, 8, 11, 11, 27, 20, 37, 18, 49, 37, 12, 75, 20, 47, 11, 26, 18, 17, 82, 9, 17, 11, 36, 16, 29, 18, 58, 20, 15, 17, 7, 40, 20, 9, 73, 15, 17, 88, 27, 48, 17, 24, 43, 15, 52, 54, 24, 20, 20, 34, 11, 13, 43, 52, 82, 7, 19, 67, 18, 54, 45, 11, 59, 64, 99, 3, 44, 18, 38, 20, 32, 45, 40, 18, 55, 83, 26, 12, 16, 36, 27, 29, 15, 16, 18, 48, 52, 11, 70, 16, 9, 20, 93, 30, 16, 54, 66, 30, 11, 22, 26, 34, 29, 17, 14, 77, 74, 13, 18, 44, 28, 56, 25, 59, 43, 20, 32, 20, 24, 6, 11, 56, 10, 17, 7, 24, 53, 46, 57, 30, 38, 36, 18, 20, 32, 78, 135, 29, 21, 80, 79, 18, 55, 20, 33, 18]


[

In [11]:
print(new_vasp_calls)

[[10, 30, 6, 38, 17, 38, 26, 20, 16], [15, 35, 6, 11, 19, 15, 17, 22, 6], [52, 33, 23, 23, 53, 24, 25, 6, 21, 22], [6, 14, 7, 17, 16, 47, 21, 23, 37, 15], [35, 8, 18, 28, 24, 19, 12, 35, 17, 48], [31, 33, 14, 10, 30, 50, 31, 31, 12, 10], [36, 6, 16, 46, 27, 16, 19, 11, 24, 38], [15, 20, 12, 17, 13, 10, 39, 38, 31, 6], [34, 43, 8, 38, 45, 27, 27, 18, 21, 20], [11, 23, 19, 9, 17, 21, 20, 26, 32, 19], [33, 46, 6, 30, 32, 14, 20, 33, 20, 6], [7, 6, 9, 24, 19, 28, 29, 24], [8, 25, 28, 16, 16, 11, 35, 59, 27], [6, 38, 21, 23, 6, 13, 22, 28, 18], [29, 30, 6, 27, 16, 6, 63, 50, 35], [29, 11, 8, 23, 20, 27, 44, 20, 39, 27], [23, 22, 23, 6, 28, 25, 21, 25, 6, 13], [26, 23, 27, 9, 6, 27, 24, 19, 41, 22], [20, 32, 24, 47, 14, 45, 16, 57, 23, 12], [8, 33, 15, 11, 23, 18, 16, 22, 30], [18, 17, 37, 29, 6, 23, 41, 22, 42, 27], [14, 13, 17, 16, 30, 26, 20, 24, 32, 22], [46, 6, 24, 51, 14, 22, 20, 33, 74, 23], [21, 15, 10, 6, 6, 26, 7, 14, 6, 14], [29, 16, 24, 26, 38, 27, 32, 16, 13, 30]]


In [12]:
print(similar_clus)

[(0, 0), (0, 2), (1, 0), (1, 2), (1, 3), (2, 7), (2, 8), (3, 0), (3, 1), (3, 2), (3, 5), (3, 7), (4, 1), (5, 0), (5, 9), (6, 1), (6, 5), (6, 9), (7, 0), (7, 1), (7, 3), (7, 5), (7, 9), (8, 2), (8, 5), (8, 9), (9, 0), (9, 1), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (10, 2), (10, 9), (11, 0), (11, 1), (11, 2), (12, 0), (12, 4), (12, 8), (13, 0), (13, 3), (13, 4), (13, 5), (14, 2), (14, 5), (15, 1), (15, 2), (16, 3), (16, 6), (16, 7), (16, 8), (16, 9), (17, 4), (17, 5), (18, 4), (19, 0), (19, 2), (19, 3), (20, 0), (20, 4), (20, 5), (21, 0), (21, 1), (21, 2), (21, 9), (22, 1), (22, 4), (23, 3), (23, 4), (23, 6), (23, 8), (24, 1)]


In [13]:
same_clus_almlp = []
same_clus_vasp = []
same_clus_diff = []
for i in range(len(similar_clus)):
    p = similar_clus[i][0]
    q = similar_clus[i][1]
    
    r = new_vasp_calls[p][q] #clus_XXX.log AL
    same_clus_almlp.append(r)
    s = vasp_calls_list[p][q] #results.log DFT
    same_clus_vasp.append(s)
    
    same_clus_diff.append(s-r)
    #print(r, s, s-r)
print(same_clus_almlp)
print(sum(same_clus_almlp))
print(same_clus_vasp)
print(sum(same_clus_vasp))
print(same_clus_diff)
print(sum(same_clus_diff))

[10, 6, 15, 6, 11, 6, 21, 6, 14, 7, 47, 23, 8, 31, 10, 6, 16, 38, 15, 20, 17, 10, 6, 8, 27, 20, 11, 23, 9, 17, 21, 20, 26, 6, 6, 7, 6, 9, 8, 16, 27, 6, 23, 6, 13, 6, 6, 11, 8, 6, 21, 25, 6, 13, 6, 27, 14, 8, 15, 11, 18, 6, 23, 14, 13, 17, 22, 6, 14, 6, 6, 7, 6, 16]
1021
[24, 9, 11, 7, 84, 7, 16, 9, 43, 11, 58, 13, 40, 63, 29, 8, 52, 42, 40, 40, 165, 25, 17, 31, 74, 15, 42, 99, 50, 15, 17, 18, 66, 7, 8, 11, 11, 27, 12, 11, 82, 9, 36, 16, 29, 7, 9, 27, 48, 11, 52, 82, 7, 19, 11, 59, 45, 12, 36, 27, 52, 9, 20, 66, 30, 11, 77, 13, 28, 6, 11, 10, 7, 46]
2337
[14, 3, -4, 1, 73, 1, -5, 3, 29, 4, 11, -10, 32, 32, 19, 2, 36, 4, 25, 20, 148, 15, 11, 23, 47, -5, 31, 76, 41, -2, -4, -2, 40, 1, 2, 4, 5, 18, 4, -5, 55, 3, 13, 10, 16, 1, 3, 16, 40, 5, 31, 57, 1, 6, 5, 32, 31, 4, 21, 16, 34, 3, -3, 52, 17, -6, 55, 7, 14, 0, 5, 3, 1, 30]
1316
